# Comibing The Articles From The Guardian, EDA and Cleaning

<img src='theguardian.jpg'>
<img src='TGuard_WordCloud.jpg'>

# Importing packages

In [31]:
import zipfile
import os
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# First file to start with
df = pd.read_json("/Users/Yemre/the_guardian/theguardian-api-python/tempdata/articles/2015-12-31.json" , encoding='uth-8')

In [3]:
# Combining all json files to a one data frame
count = 0
for filename in os.listdir("/Users/Yemre/the_guardian/theguardian-api-python/tempdata/news_of_interest/"):
    count+=1
    if count>2:
        file_path = "/Users/Yemre/the_guardian/theguardian-api-python/tempdata/articles/" + filename
        df_ = pd.read_json(file_path , encoding='uth-8')
        df = pd.concat(objs= [df,df_], axis=0,ignore_index=True)
        
        # To see the progress, print the number of the file
        if count%100 ==0:
            print "This is the ",count,"th file -->"

This is the  100 th file -->
This is the  200 th file -->
This is the  300 th file -->
This is the  400 th file -->


In [4]:
# Getting bodytext from the dataframe
lst = []
for i in range(df.shape[0]):
    lst.append(df.fields[i]["bodyText"])

In [5]:
# Creating a column with bodytext
df["bodyText"] = lst

In [8]:
# Getting headlines from the dataframe
lst_head = []
for i in range(df.shape[0]):
        lst_head.append(df.fields[i]["headline"])

In [9]:
# Creating a column with headline
df["headline"] = lst_head

In [11]:
# Getting rid of the embty bodies
df = df[df.bodyText != ""]

In [13]:
# Total Number of Articles
df.shape

(129107, 12)

In [15]:
# Filtering articles based on topics
df = df[(df.sectionName == 'US news') | (df.sectionName == 'Business') | (df.sectionName == 'Politics') | (df.sectionName == 'World news')]

In [17]:
# Getting to see the dataframe info (data type, non-null value etc.)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27795 entries, 7 to 130903
Data columns (total 12 columns):
apiUrl                27795 non-null object
fields                27795 non-null object
id                    27795 non-null object
isHosted              27795 non-null bool
sectionId             27795 non-null object
sectionName           27795 non-null object
type                  27795 non-null object
webPublicationDate    27795 non-null object
webTitle              27795 non-null object
webUrl                27795 non-null object
bodyText              27795 non-null object
headline              27795 non-null object
dtypes: bool(1), object(11)
memory usage: 2.6+ MB


In [18]:
# To see how many articles in each category
Counter(df.sectionName)

Counter({u'Business': 6079,
         u'Politics': 5777,
         u'US news': 6707,
         u'World news': 9232})

In [19]:
# Overview for the dataframe
df.head()

,apiUrl,fields,id,isHosted,sectionId,sectionName,type,webPublicationDate,webTitle,webUrl,bodyText,headline
7,https://content.guardianapis.com/business/2015...,{u'bodyText': u'Stock markets around the world...,business/2015/dec/31/value-of-gun-manufacturer...,False,business,Business,article,2015-12-31T21:33:28Z,Value of gun manufacturers' stocks almost doub...,https://www.theguardian.com/business/2015/dec/...,Stock markets around the world closed down on ...,Value of gun manufacturers' stocks almost doub...
16,https://content.guardianapis.com/us-news/2015/...,{u'bodyText': u'Young black men were nine time...,us-news/2015/dec/31/the-counted-police-killing...,False,us-news,US news,article,2015-12-31T20:00:20Z,Young black men killed by US police at highest...,https://www.theguardian.com/us-news/2015/dec/3...,Young black men were nine times more likely th...,Young black men killed by US police at highest...
17,https://content.guardianapis.com/us-news/2015/...,"{u'bodyText': u'At the beginning of 2015, the ...",us-news/2015/dec/31/the-counted-killings-by-po...,False,us-news,US news,article,2015-12-31T20:00:20Z,The Guardian view on killings by US police: wh...,https://www.theguardian.com/us-news/2015/dec/3...,"At the beginning of 2015, the Guardian began a...",The Guardian view on killings by US police: wh...
18,https://content.guardianapis.com/us-news/2015/...,{u'bodyText': u'Environmentalists will ramp up...,us-news/2015/dec/31/animal-themed-condoms-slow...,False,us-news,US news,article,2015-12-31T19:55:06Z,Animal-themed condoms aim to slow population g...,https://www.theguardian.com/us-news/2015/dec/3...,Environmentalists will ramp up their attempts ...,Animal-themed condoms aim to slow population g...
20,https://content.guardianapis.com/world/2015/de...,{u'bodyText': u'Letters written by a former ar...,world/2015/dec/31/archbishop-and-mps-wrote-in-...,False,world,World news,article,2015-12-31T19:45:22Z,Archbishop and MPs wrote in support of bishop ...,https://www.theguardian.com/world/2015/dec/31/...,Letters written by a former archbishop of Cant...,Archbishop and MPs wrote in support of bishop ...


In [22]:
# Publication data range
df.webPublicationDate.min() ,df.webPublicationDate.max()

(u'2015-12-31T00:01:21Z', u'2017-03-27T22:26:20Z')

In [75]:
for idx,item in enumerate(df_test.bodyText):
    df_test.bodyText[idx] = re.sub('[^\x00-\x7F]+', "", item)
    if idx%500 == 0:
        print 'Here is the ',idx,'th item'

/Users/Yemre/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Here is the  0 th item
Here is the  500 th item
Here is the  1000 th item
Here is the  1500 th item
Here is the  2000 th item
Here is the  2500 th item
Here is the  3000 th item
Here is the  3500 th item
Here is the  4000 th item
Here is the  4500 th item
Here is the  5000 th item
Here is the  5500 th item
Here is the  6000 th item
Here is the  6500 th item
Here is the  7000 th item
Here is the  7500 th item
Here is the  8000 th item
Here is the  8500 th item
Here is the  9000 th item
Here is the  9500 th item
Here is the  10000 th item
Here is the  10500 th item
Here is the  11000 th item
Here is the  11500 th item
Here is the  12000 th item
Here is the  12500 th item
Here is the  13000 th item
Here is the  13500 th item
Here is the  14000 th item
Here is the  14500 th item
Here is the  15000 th item
Here is the  15500 th item
Here is the  16000 th item
Here is the  16500 th item
Here is the  17000 th item
Here is the  17500 th item
Here is the  18000 th item
Here is the  18500 th ite

In [76]:
for idx,item in enumerate(df_test.headline):
    df_test.headline[idx] = re.sub('[^\x00-\x7F]+', "", item)
    if idx%500 == 0:
        print 'Here is the ',idx,'th item'

/Users/Yemre/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Here is the  0 th item
Here is the  500 th item
Here is the  1000 th item
Here is the  1500 th item
Here is the  2000 th item
Here is the  2500 th item
Here is the  3000 th item
Here is the  3500 th item
Here is the  4000 th item
Here is the  4500 th item
Here is the  5000 th item
Here is the  5500 th item
Here is the  6000 th item
Here is the  6500 th item
Here is the  7000 th item
Here is the  7500 th item
Here is the  8000 th item
Here is the  8500 th item
Here is the  9000 th item
Here is the  9500 th item
Here is the  10000 th item
Here is the  10500 th item
Here is the  11000 th item
Here is the  11500 th item
Here is the  12000 th item
Here is the  12500 th item
Here is the  13000 th item
Here is the  13500 th item
Here is the  14000 th item
Here is the  14500 th item
Here is the  15000 th item
Here is the  15500 th item
Here is the  16000 th item
Here is the  16500 th item
Here is the  17000 th item
Here is the  17500 th item
Here is the  18000 th item
Here is the  18500 th ite

In [79]:
for idx, item in enumerate(df_test.bodyText):
    df_test.bodyText[idx] = re.sub('(\\n)',"",item)
    if idx%500 == 0:
        print 'Here is the ',idx,'th item'

/Users/Yemre/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Here is the  0 th item
Here is the  500 th item
Here is the  1000 th item
Here is the  1500 th item
Here is the  2000 th item
Here is the  2500 th item
Here is the  3000 th item
Here is the  3500 th item
Here is the  4000 th item
Here is the  4500 th item
Here is the  5000 th item
Here is the  5500 th item
Here is the  6000 th item
Here is the  6500 th item
Here is the  7000 th item
Here is the  7500 th item
Here is the  8000 th item
Here is the  8500 th item
Here is the  9000 th item
Here is the  9500 th item
Here is the  10000 th item
Here is the  10500 th item
Here is the  11000 th item
Here is the  11500 th item
Here is the  12000 th item
Here is the  12500 th item
Here is the  13000 th item
Here is the  13500 th item
Here is the  14000 th item
Here is the  14500 th item
Here is the  15000 th item
Here is the  15500 th item
Here is the  16000 th item
Here is the  16500 th item
Here is the  17000 th item
Here is the  17500 th item
Here is the  18000 th item
Here is the  18500 th ite

In [80]:
for idx, item in enumerate(df_test.headline):
    df_test.headline[idx] = re.sub('(\\n)',"",item)
    if idx%500 == 0:
        print 'Here is the ',idx,'th item'

/Users/Yemre/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Here is the  0 th item
Here is the  500 th item
Here is the  1000 th item
Here is the  1500 th item
Here is the  2000 th item
Here is the  2500 th item
Here is the  3000 th item
Here is the  3500 th item
Here is the  4000 th item
Here is the  4500 th item
Here is the  5000 th item
Here is the  5500 th item
Here is the  6000 th item
Here is the  6500 th item
Here is the  7000 th item
Here is the  7500 th item
Here is the  8000 th item
Here is the  8500 th item
Here is the  9000 th item
Here is the  9500 th item
Here is the  10000 th item
Here is the  10500 th item
Here is the  11000 th item
Here is the  11500 th item
Here is the  12000 th item
Here is the  12500 th item
Here is the  13000 th item
Here is the  13500 th item
Here is the  14000 th item
Here is the  14500 th item
Here is the  15000 th item
Here is the  15500 th item
Here is the  16000 th item
Here is the  16500 th item
Here is the  17000 th item
Here is the  17500 th item
Here is the  18000 th item
Here is the  18500 th ite

In [81]:
# Saving the cleaned data into csv file
df_test.to_csv("/Users/Yemre/Desktop/Clean_TheGuardian_Combined_No_Slash.csv")

In [3]:
# Reading the cleaned data into data frame
df_test = pd.read_csv("/Users/Yemre/Desktop/Clean_TheGuardian_Combined_No_Slash.csv")

In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27795 entries, 0 to 27794
Data columns (total 14 columns):
Unnamed: 0            27795 non-null int64
Unnamed: 0.1          27795 non-null int64
apiUrl                27795 non-null object
fields                27795 non-null object
id                    27795 non-null object
isHosted              27795 non-null bool
sectionId             27795 non-null object
sectionName           27795 non-null object
type                  27795 non-null object
webPublicationDate    27795 non-null object
webTitle              27795 non-null object
webUrl                27795 non-null object
bodyText              27795 non-null object
headline              27795 non-null object
dtypes: bool(1), int64(2), object(11)
memory usage: 2.8+ MB


In [24]:
sum_ = 0
max_ = 0
min_ = 9999
for item in df.bodyText:
    sum_+=len(item)
    if len(item)>max_:
        max_ = len(item)
    if len(item)<min:
        min_ = len(item)

In [30]:
print "Min number of word count",min_ ,"\nMax number of word count",max_,"\nTotal number of word count", sum_

Min number of word count 4967 
Max number of word count 174383 
Total number of word count 165783149


# Word Clound From The Guardian Dataset

<img src='TGuard_WordCloud.jpg'>